In [1]:
import cv2, sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib tk

from scipy import linalg


In [2]:
# by https://temugeb.github.io/opencv/python/2021/02/02/stereo-camera-calibration-and-triangulation.html
def DLT(P1, P2, point1, point2):
    A = np.array([point1[1]*P1[2,:]-P1[1,:], P1[0,:]-point1[0]*P1[2,:], point2[1]*P2[2,:]-P2[1,:], P2[0,:]-point2[0]*P2[2,:]]).reshape((4,4))
    B = A.transpose() @ A
    U, s, Vh = linalg.svd(B, full_matrices = False)
    return Vh[3,0:3]/Vh[3,3]


In [3]:

# convention marker sort: left to right (x) then top to bottom (y)
# note that y top denotes to the smallest y because in the image the y axis is inversed+
# Therefore, in the images the top left point is (0,0,0) in the first frame

# parameter
N_t = np.arange(1,31) # manuel input the number of frames per camera
marker_distance, Z0 = (120, 120), 0 # X[mm], Y[mm], Z0[mm] , shift -360 in x and +360 in y for davis
marker_size = (8, 7)  # x, y
image_size = (2560, 2048)  # x[px], y[px]

# define first plane XYZ
X, Y, Z = np.meshgrid(np.arange(0,marker_size[0]*marker_distance[0],marker_distance[0]),-np.arange(0,marker_size[1]*marker_distance[1],marker_distance[1]),np.linspace(Z0,Z0,1))
XYZ = np.asarray(np.vstack([X.ravel(),Y.ravel(),Z.ravel()]).T, dtype=np.float32)

# calibrate camera 0
XYZ_0 = [XYZ for t in N_t]
xy_0 = [np.asarray(np.loadtxt('markers_xy/c{cam}/c{cam}_{time}.txt'.format(cam=0,time=str(t).zfill(5)),skiprows=1), dtype=np.float32) for t in N_t]
ret_0, M_0, d_0, r_0, t_0 = cv2.calibrateCamera(XYZ_0,xy_0,image_size,None,None) 
R_0 = cv2.Rodrigues(r_0[0])[0]    
pos_c0 = -np.dot(R_0.T, t_0[0]).ravel()
print('position cam 0: ', pos_c0)
# calibrate camera 1
XYZ_1 = [XYZ for t in N_t]
xy_1 = [np.asarray(np.loadtxt('markers_xy/c{cam}/c{cam}_{time}.txt'.format(cam=1,time=str(t).zfill(5)),skiprows=1), dtype=np.float32) for t in N_t]
ret_1, M_1, d_1, r_1, t_1 = cv2.calibrateCamera(XYZ_1,xy_1,image_size,None,None) 
R_1 = cv2.Rodrigues(r_1[0])[0]    
pos_c1 = -np.dot(R_1.T, t_1[0]).ravel()
print('position cam 1: ', pos_c1)
# calibrate camera 2
XYZ_2 = [XYZ for t in N_t]
xy_2 = [np.asarray(np.loadtxt('markers_xy/c{cam}/c{cam}_{time}.txt'.format(cam=2,time=str(t).zfill(5)),skiprows=1), dtype=np.float32) for t in N_t]
ret_2, M_2, d_2, r_2, t_2 = cv2.calibrateCamera(XYZ_2,xy_2,image_size,None,None) 
R_2 = cv2.Rodrigues(r_2[0])[0]    
pos_c2 = -np.dot(R_2.T, t_2[0]).ravel()
print('position cam 2: ', pos_c2)
# calibrate camera 3
XYZ_3 = [XYZ for t in N_t]
xy_3 = [np.asarray(np.loadtxt('markers_xy/c{cam}/c{cam}_{time}.txt'.format(cam=3,time=str(t).zfill(5)),skiprows=1), dtype=np.float32) for t in N_t]
ret_3, M_3, d_3, r_3, t_3 = cv2.calibrateCamera(XYZ_3,xy_3,image_size,None,None) 
R_3 = cv2.Rodrigues(r_3[0])[0]    
pos_c3 = -np.dot(R_3.T, t_3[0]).ravel()
print('position cam 3: ', pos_c3)

# stereo matching of the marker positons - use only cam 0 and cam 1
ret, CM0, dist0, CM1, dist1, R, T, E, F = cv2.stereoCalibrate(XYZ_0[0:1], xy_0[0:1], xy_1[0:1], M_0, d_0, M_1, d_1, image_size)
#projection matrix for camera 0
RT0 = np.concatenate([R_0, t_0[0]], axis = -1)
P0 = M_0 @ RT0 
#projection matrix for camera 1
RT1 = np.concatenate([R@R_0, (R@t_0[0]+T)], axis = -1)
P1 = M_1 @ RT1 
# P contains the 3D marker positions of each plate at each position
P = []
for i in range(len(N_t)):
    xyz = []
    for xy0, xy1 in zip(xy_0[i],xy_1[i]):
        xyz.append(DLT(P0, P1, xy0, xy1))
    P.append(np.asarray(xyz,dtype=np.float32))

# recalibrate camera 0
XYZ_0 = [np.asarray(Pj,dtype=np.float32) for Pj in P]
xy_0 = [np.asarray(np.loadtxt('markers_xy/c{cam}/c{cam}_{time}.txt'.format(cam=0,time=str(t).zfill(5)),skiprows=1), dtype=np.float32) for t in N_t]
ret_0, M_0, d_0, r_0, t_0 = cv2.calibrateCamera(XYZ_0,xy_0,image_size,M_0,d_0,flags=cv2.CALIB_USE_INTRINSIC_GUESS)
R_0 = cv2.Rodrigues(r_0[0])[0]    
pos_c0_0 = -np.dot(R_0.T, t_0[0]).ravel()
print('new position cam 0: ', pos_c0_0)


position cam 0:  [ 1979.99310731 -1885.97837947  3898.61614005]
position cam 1:  [2028.5582441   -14.248985   3917.15644191]
position cam 2:  [ -836.29003268 -1893.56968485  3998.66845662]
position cam 3:  [-808.92550278  -21.82376922 4033.34571974]
new position cam 0:  [ 2003.59767018 -1872.80392155  3908.79140332]


In [4]:

# recalibrate camera 1
XYZ_1 = [np.asarray(Pj,dtype=np.float32) for Pj in P]
xy_1 = [np.asarray(np.loadtxt('markers_xy/c{cam}/c{cam}_{time}.txt'.format(cam=1,time=str(t).zfill(5)),skiprows=1), dtype=np.float32) for t in N_t]
ret_1, M_1, d_1, r_1, t_1 = cv2.calibrateCamera(XYZ_1,xy_1,image_size,M_1,d_1,flags=cv2.CALIB_USE_INTRINSIC_GUESS) 
# recalibrate camera 2
XYZ_2 = [np.asarray(Pj,dtype=np.float32) for Pj in P]
xy_2 = [np.asarray(np.loadtxt('markers_xy/c{cam}/c{cam}_{time}.txt'.format(cam=2,time=str(t).zfill(5)),skiprows=1), dtype=np.float32) for t in N_t]
ret_2, M_2, d_2, r_2, t_2 = cv2.calibrateCamera(XYZ_2,xy_2,image_size,M_2,d_2,flags=cv2.CALIB_USE_INTRINSIC_GUESS) 
# recalibrate camera 3
XYZ_3 = [np.asarray(Pj,dtype=np.float32) for Pj in P]
xy_3 = [np.asarray(np.loadtxt('markers_xy/c{cam}/c{cam}_{time}.txt'.format(cam=3,time=str(t).zfill(5)),skiprows=1), dtype=np.float32) for t in N_t]
ret_3, M_3, d_3, r_3, t_3 = cv2.calibrateCamera(XYZ_3,xy_3,image_size,M_3,d_3,flags=cv2.CALIB_USE_INTRINSIC_GUESS) 





# plot 3D position
fig = plt.figure(figsize=(12,12))
axis = fig.add_subplot(111, projection='3d')
axis.set_xlabel('Z [mm]'), axis.set_ylabel('X [mm]'), axis.set_zlabel('Y [mm]')
axis.set_xlim(-3000,5000), axis.set_ylim(-3500,4500), axis.set_zlim(-2150,250) # 2.38 x 7.0 m , Z=0 -> 4.5 # 20cm bis oben und unten
axis.scatter(pos_c0[2],pos_c0[0],pos_c0[1],label='c0')  
axis.scatter(pos_c1[2],pos_c1[0],pos_c1[1],label='c1')  
axis.scatter(pos_c2[2],pos_c2[0],pos_c2[1],label='c2')  
axis.scatter(pos_c3[2],pos_c3[0],pos_c3[1],label='c3')  
axis.scatter(XYZ[:,2],XYZ[:,0],XYZ[:,1],c='red',label='first plane')   
[axis.scatter(P[i][:,2],P[i][:,0],P[i][:,1],c='green') for i in range(1,len(N_t))]
theta = np.linspace(0, 2*np.pi, 100)
geometry_down = [3500*np.cos(theta)+1000,3500*np.sin(theta)+480,np.zeros_like(theta)-2150]
geometry_up = [3500*np.cos(theta)+1000,3500*np.sin(theta)+480,np.zeros_like(theta)+250]
axis.plot(geometry_down[0],geometry_down[1],geometry_down[2],c='black')
axis.plot(geometry_up[0],geometry_up[1],geometry_up[2],c='black')
axis.plot(1000,480,-2150,'x',c='black')
axis.plot(1000,480,250,'x',c='black')
plt.legend()
plt.show()

'''
# plot 2D position of cam 0 at time 1
p, _ = cv2.projectPoints(P[0], r_0[0], t_0[0], M_0, d_0)
p = p.reshape(56,2)
img = cv2.imread("../../Calibration/c{cam}_{time}.tif".format(cam=0,time=str(1).zfill(5)),cv2.IMREAD_UNCHANGED)
plt.figure()
plt.imshow(img,cmap='gray')
plt.plot(p[:,0],p[:,1],'o',c='red')
plt.plot(xy_0[0][:,0],xy_0[0][:,1],'o',c='blue')
plt.show()

# plot 2D position of cam 0 at time 2
p, _ = cv2.projectPoints(P[1], r_0[1], t_0[1], M_0, d_0)
p = p.reshape(56,2)
img = cv2.imread("../../Calibration/c{cam}_{time}.tif".format(cam=0,time=str(2).zfill(5)),cv2.IMREAD_UNCHANGED)
plt.figure()
plt.imshow(img,cmap='gray')
plt.plot(p[:,0],p[:,1],'o',c='red')
plt.plot(xy_0[1][:,0],xy_0[1][:,1],'o',c='blue')
plt.show()
'''


'\n# plot 2D position of cam 0 at time 1\np, _ = cv2.projectPoints(P[0], r_0[0], t_0[0], M_0, d_0)\np = p.reshape(56,2)\nimg = cv2.imread("../../Calibration/c{cam}_{time}.tif".format(cam=0,time=str(1).zfill(5)),cv2.IMREAD_UNCHANGED)\nplt.figure()\nplt.imshow(img,cmap=\'gray\')\nplt.plot(p[:,0],p[:,1],\'o\',c=\'red\')\nplt.plot(xy_0[0][:,0],xy_0[0][:,1],\'o\',c=\'blue\')\nplt.show()\n\n# plot 2D position of cam 0 at time 2\np, _ = cv2.projectPoints(P[1], r_0[1], t_0[1], M_0, d_0)\np = p.reshape(56,2)\nimg = cv2.imread("../../Calibration/c{cam}_{time}.tif".format(cam=0,time=str(2).zfill(5)),cv2.IMREAD_UNCHANGED)\nplt.figure()\nplt.imshow(img,cmap=\'gray\')\nplt.plot(p[:,0],p[:,1],\'o\',c=\'red\')\nplt.plot(xy_0[1][:,0],xy_0[1][:,1],\'o\',c=\'blue\')\nplt.show()\n'

In [5]:
# Generate new_XYZ set of 3D points
P_array = np.vstack(P)
# new_XYZ = np.array(np.meshgrid(
#     np.linspace(P_array[:, 0].min(), P_array[:, 0].max(), 5),
#     np.linspace(P_array[:, 1].min(), P_array[:, 1].max(), 4),
#     np.linspace(P_array[:, 2].min(), P_array[:, 2].max(), 3)
# )).T.reshape(-1, 3)


x = np.linspace(P_array[:, 0].min(), P_array[:, 0].max(), 3)
y = np.linspace(P_array[:, 1].min(), P_array[:, 1].max(), 3)
z = np.linspace(P_array[:, 2].min(), P_array[:, 2].max(), 3)

X, Y, Z = np.meshgrid(x, y, z)
new_XYZ = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T



# Create images for each calibrated camera
for cam_idx, (r, t, M, d) in enumerate(zip([r_0, r_1, r_2, r_3], [t_0, t_1, t_2, t_3], [M_0, M_1, M_2, M_3], [d_0, d_1, d_2, d_3])):
    imgpoints, _ = cv2.projectPoints(new_XYZ, r[0], t[0], M, d)
    binary_image = np.zeros((image_size[1], image_size[0]), dtype=np.uint8)
    
    for point in imgpoints:
        cv2.circle(binary_image, (int(point[0][0]), int(point[0][1])), 7, (255, 255, 255), -1)
        cv2.imwrite(f'cam{cam_idx+1}.tif', binary_image)
    
    plt.figure()
    plt.title(f'Camera {cam_idx}')
    plt.imshow(binary_image, cmap='gray')
    plt.show()

In [6]:
import pandas as pd

# Create a DataFrame with particle numbers and positions
particle_numbers = np.arange(1, len(new_XYZ) + 1)
# Reshape new_XYZ to match the order of coordinates
# new_XYZ_reshaped = new_XYZ.reshape(5, 4, 3, 3).transpose(2, 1, 0, 3).reshape(-1, 3)
data = np.column_stack((particle_numbers, new_XYZ))
df_new_XYZ = pd.DataFrame(data, columns=['Particle', 'X', 'Y', 'Z'])

# Save to a tab-delimited CSV file
df_new_XYZ.to_csv('new_XYZ.csv', sep='\t', index=False, header=False)

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot
ax.scatter(new_XYZ[:, 0], new_XYZ[:, 1], new_XYZ[:, 2], c='b', marker='o')

# Annotate each point with its particle number
for i, txt in enumerate(particle_numbers):
    ax.text(new_XYZ[i, 0], new_XYZ[i, 1], new_XYZ[i, 2], '%d' % txt, size=10, zorder=1, color='k')

# Set labels
ax.set_xlabel('X (left to right)')
ax.set_ylabel('Y (top-down)')
ax.set_zlabel('Z (into the page)')

# Adjust the view angle
ax.view_init(elev=90, azim=-90)

plt.show()

invalid command name "140324181394816delayed_destroy"
    while executing
"140324181394816delayed_destroy"
    ("after" script)
invalid command name "140324181373376delayed_destroy"
    while executing
"140324181373376delayed_destroy"
    ("after" script)
